## 데이터 수집 : 항공 스케줄 데이터 크롤링
---
* 출처 : [항공정보포털시스템](https://www.airportal.go.kr/index.jsp)
* 수집 범위 
    + 기간 : 2017년 1월 ~ 2022년 2월
    + 출발공항 : 제주공항
    + 도착공항 : 김포, 김해, 광주, 청주 공항
    + 용도 : 여객
    + 항공사 : 대한항공
* 방법 : selenium을 이용한 크롤링
---
### 사용 패키지 및 드라이버
* Selenium 
    + 자동화된 웹 테스트를 수행하는 프레임워크
* Chrome Driver
    + 테스트 브라우저 드라이버
    + 버전 : 108.0.5359.71 (현재 사용 중인 Chrome 브라우저의 버전을 따름) 
---
#### *Selenium의 단점인 속도를 보완하기 위하여 반복문을 사용하지 않음
#### 반복문을 사용하지 않았기 때문에 소스코드를 반복하여 나타내지 않았음(이 노트북 문서에는 일부만 포함됨)
#### 아래의 소스코드를 반복하여 실행

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import csv
import time

# selemium 드라이버를 크롬버전에 맞추어 자동으로 업데이트
driver = wd.Chrome(service = Service(ChromeDriverManager().install()))
# 크롤링 할 url 지정
url = 'https://www.airportal.go.kr/life/airinfo/RaSkeFrmMain.jsp'
# 웹 페이지 자동 오픈
driver.get(url)

#출발 공항 선택
driver.switch_to.frame('main')
driver.switch_to.frame('airportframe')
driver.find_element_by_xpath('//*[@name="ser_airport"]/option[text()="제주"]').click()

#도착 공항 선택
driver.switch_to.parent_frame()
driver.switch_to.frame(1)
driver.find_element_by_xpath('//*[@name="ser_region"]/option[text()="동북아시아"]').click()
driver.find_element_by_xpath('//*[@name="ser_nation"]/option[text()="한국"]').click()
driver.find_element_by_xpath('//*[@name="ser_airport"]/option[text()="김포"]').click()

#기간 선택(from)
driver.switch_to.parent_frame()
driver.find_element_by_name('current_dt_from').clear()
driver.find_element_by_name('current_dt_from').send_keys('20170101')
        
#기간 선택(to)
driver.find_element_by_name('current_dt_to').clear()
driver.find_element_by_name('current_dt_to').send_keys('20170131')
        
#용도 선택
driver.find_element_by_xpath('//*[@name="regCls"]/option[text()="여객"]').click()

#항공사 선택
driver.find_element_by_xpath('//*[@name="al_icao"]/option[text()="대한항공"]').click()

#결과 크롤링
driver.switch_to.parent_frame()
driver.switch_to.frame('main')
driver.switch_to.frame('sframe')
tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
tr = tbody.find_elements_by_tag_name('tr')
num = int(driver.find_element_by_xpath('/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[4]/td/table/tbody/tr/td[3]/font[1]').text)
if(num % 10 == 0):
    pages = num // 10
else :
    pages = num // 10 + 1
print("----------------------------%d 건, %d 페이지----------------------------"%(num, pages))
col_name = ["일자", "공백1", "공백2", "항공사", "편명", "도착", "계획", "예상", "출발", "구분", "현황"] 
        

with open('2017_1', 'w', -1, newline= '') as f :
    w = csv.writer(f)
    w.writerow(col_name)
    for p in range(pages // 10) :
        for n in range(11):
            # 1장 1페이지
            if (p == 0) & (n == 0) :
                tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                tr = tbody.find_elements_by_tag_name('tr')
                for td in tr:
                    row = td.text
                    row_list = row.split(' ')
                    print(row_list)
                    w.writerow(row_list)
                    time.sleep(0.1)
            else :   
                #1쪽이 아닌 경우, next 버튼 누르기
                if (p != 0) &(n == 10) :
                    driver.find_element_by_xpath('/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[4]/td/table/tbody/tr/td[2]/font/font/a[10]/font/img').click()
                    tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                    tr = tbody.find_elements_by_tag_name('tr')
                    for td in tr:
                        row = td.text
                        row_list = row.split(' ')
                        print(row_list)
                        w.writerow(row_list)
                        time.sleep(0.1)
                else : 
                    # 첫번째 페이지
                    if n == 0 : 
                        tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                        tr = tbody.find_elements_by_tag_name('tr')
                        for td in tr:
                            row = td.text
                            row_list = row.split(' ')
                            print(row_list)
                            w.writerow(row_list)
                            time.sleep(0.1)
                    else : 
                        #1쪽인 경우 
                        if p == 0 : 
                            xpath1 = '/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[4]/td/table/tbody/tr/td[2]/font/a['
                            xpath2 = ']/font'
                            xpath = xpath1 + str(n) + xpath2
                            driver.find_element_by_xpath(xpath).click()
                                    
                            tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                            tr = tbody.find_elements_by_tag_name('tr')
                            for td in tr:
                                row = td.text
                                row_list = row.split(' ')
                                print(row_list)
                                w.writerow(row_list)
                                time.sleep(0.1)
                        else : 
                            xpath1 = '/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[4]/td/table/tbody/tr/td[2]/font/font/a['
                            xpath2 = ']/font'
                            xpath = xpath1 + str(n) + xpath2
                            driver.find_element_by_xpath(xpath).click()
                                    
                            tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                            tr = tbody.find_elements_by_tag_name('tr')
                            for td in tr:
                                row = td.text
                                row_list = row.split(' ')
                                print(row_list)
                                w.writerow(row_list)
                                time.sleep(0.1)
    print(pages % 10)
    print(num%10)
    if(pages%10 != 0):
        for a in range(pages%10 -1):
            for n in range(11):  
                if n == 0 : 
                    tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                    tr = tbody.find_elements_by_tag_name('tr')
                    for td in tr:
                        row= td.text
                        row_list = row.split(' ')
                        print(row_list)
                        w.writerow(row_list)
                        time.sleep(0.1)
                else :  
                    xpath1 = '/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[4]/td/table/tbody/tr/td[2]/font/font/a['
                    xpath2 = ']/font'
                    xpath = xpath1 + str(n) + xpath2
                    driver.find_element_by_xpath(xpath).click()
                    tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')
                    tr = tbody.find_elements_by_tag_name('tr')
                    for td in tr:
                        row = td.text
                        row_list = row.split(' ')
                        print(row_list)
                        w.writerow(row_list)
                        time.sleep(0.1) 


C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@name="ser_airport"]/option[text()="제주"]').click()
C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@name="ser_region"]/option[text()="동북아시아"]').click()
C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:24: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@name="ser_nation"]/option[text()="한국"]').click()
C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.f

----------------------------721 건, 73 페이지----------------------------
['2017-01-31', '', '', '대한항공', 'KE1200', '제주(CJU)->김포(GMP)', '06:55', '06:55', '07:12', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1250', '제주(CJU)->김포(GMP)', '07:10', '07:10', '07:27', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1204', '제주(CJU)->김포(GMP)', '10:25', '10:25', '10:45', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1210', '제주(CJU)->김포(GMP)', '11:10', '11:10', '11:26', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1212', '제주(CJU)->김포(GMP)', '11:40', '11:40', '11:56', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1214', '제주(CJU)->김포(GMP)', '12:10', '12:10', '12:24', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1256', '제주(CJU)->김포(GMP)', '12:40', '12:40', '12:58', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1216', '제주(CJU)->김포(GMP)', '13:15', '14:01', '13:58', '여객', '지연']
['']
['2017-01-31', '', '', '대한항공', 'KE1218', '제주(CJU)->김포(GMP)', '13:35', '13:35', '14:02', '여

C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:101: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(xpath).click()
C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:103: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')


['2017-01-31', '', '', '대한항공', 'KE1222', '제주(CJU)->김포(GMP)', '15:05', '15:05', '15:24', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1254', '제주(CJU)->김포(GMP)', '15:15', '15:15', '15:44', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1224', '제주(CJU)->김포(GMP)', '15:30', '16:00', '16:16', '여객', '지연']
['']
['2017-01-31', '', '', '대한항공', 'KE1226', '제주(CJU)->김포(GMP)', '16:30', '16:30', '16:40', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1232', '제주(CJU)->김포(GMP)', '17:45', '17:45', '18:03', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1234', '제주(CJU)->김포(GMP)', '18:40', '19:09', '19:10', '여객', '지연']
['']
['2017-01-31', '', '', '대한항공', 'KE1236', '제주(CJU)->김포(GMP)', '19:00', '19:30', '19:30', '여객', '지연']
['']
['2017-01-31', '', '', '대한항공', 'KE1238', '제주(CJU)->김포(GMP)', '19:30', '19:30', '19:47', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1240', '제주(CJU)->김포(GMP)', '20:10', '20:10', '20:27', '여객', '출발']
['']
['2017-01-31', '', '', '대한항공', 'KE1242', '제주(CJU)->김포(G

['2017-01-28', '', '', '대한항공', 'KE1232', '제주(CJU)->김포(GMP)', '17:45', '17:45', '18:09', '여객', '출발']
['']
['2017-01-28', '', '', '대한항공', 'KE1234', '제주(CJU)->김포(GMP)', '18:40', '18:40', '18:55', '여객', '출발']
['']
['2017-01-28', '', '', '대한항공', 'KE1236', '제주(CJU)->김포(GMP)', '19:00', '19:32', '19:31', '여객', '지연']
['']
['2017-01-28', '', '', '대한항공', 'KE1238', '제주(CJU)->김포(GMP)', '19:30', '19:30', '19:44', '여객', '출발']
['']
['2017-01-28', '', '', '대한항공', 'KE1240', '제주(CJU)->김포(GMP)', '20:10', '20:10', '20:20', '여객', '출발']
['']
['2017-01-28', '', '', '대한항공', 'KE1244', '제주(CJU)->김포(GMP)', '20:55', '20:55', '21:06', '여객', '출발']
['']
['2017-01-28', '', '', '대한항공', 'KE1246', '제주(CJU)->김포(GMP)', '21:05', '21:05', '21:16', '여객', '출발']
['']
['2017-01-27', '', '', '대한항공', 'KE1282', '제주(CJU)->김포(GMP)', '06:30', '06:30', '06:42', '여객', '출발']
['']
['2017-01-27', '', '', '대한항공', 'KE1200', '제주(CJU)->김포(GMP)', '06:55', '06:55', '07:07', '여객', '출발']
['']
['2017-01-27', '', '', '대한항공', 'KE1250', '제주(CJU)->김포(G

...

C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:131: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')


['2017-01-01', '', '', '대한항공', 'KE1210', '제주(CJU)->김포(GMP)', '11:10', '11:10', '11:30', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1254', '제주(CJU)->김포(GMP)', '11:35', '11:35', '11:47', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1212', '제주(CJU)->김포(GMP)', '11:40', '11:40', '11:57', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1214', '제주(CJU)->김포(GMP)', '12:10', '12:10', '12:25', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1216', '제주(CJU)->김포(GMP)', '13:15', '13:15', '13:37', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1218', '제주(CJU)->김포(GMP)', '13:35', '13:35', '13:56', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1248', '제주(CJU)->김포(GMP)', '14:15', '14:15', '14:36', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1222', '제주(CJU)->김포(GMP)', '15:00', '15:00', '15:25', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1224', '제주(CJU)->김포(GMP)', '15:30', '15:30', '15:50', '여객', '출발']
['']


C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:143: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(xpath).click()
C:\Users\yeji4\AppData\Local\Temp/ipykernel_1732/3771746453.py:144: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbody = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[3]/td/table/tbody')


['2017-01-01', '', '', '대한항공', 'KE1226', '제주(CJU)->김포(GMP)', '16:30', '16:30', '16:59', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1228', '제주(CJU)->김포(GMP)', '16:50', '16:50', '17:13', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1220', '제주(CJU)->김포(GMP)', '17:20', '17:20', '17:41', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1232', '제주(CJU)->김포(GMP)', '17:45', '18:05', '18:29', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1234', '제주(CJU)->김포(GMP)', '18:40', '19:13', '19:13', '여객', '지연']
['']
['2017-01-01', '', '', '대한항공', 'KE1236', '제주(CJU)->김포(GMP)', '19:00', '19:00', '19:24', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1238', '제주(CJU)->김포(GMP)', '19:40', '19:40', '19:55', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1240', '제주(CJU)->김포(GMP)', '20:10', '20:10', '20:28', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1242', '제주(CJU)->김포(GMP)', '20:45', '20:45', '21:08', '여객', '출발']
['']
['2017-01-01', '', '', '대한항공', 'KE1244', '제주(CJU)->김포(G

ElementClickInterceptedException: Message: element click intercepted: Element <iframe name="sframe" src="/life/airinfo/RaSkeDepList.jsp" width="670" height="490" frameborder="0" cd_frame_id_="a5d6ff9851fc03b3a38ba80a8f7b7c9c"></iframe> is not clickable at point (343, 312). Other element would receive the click: <a class="ui-state-default" href="#">...</a>
  (Session info: chrome=108.0.5359.95)
Stacktrace:
Backtrace:
	(No symbol) [0x0101F243]
	(No symbol) [0x00FA7FD1]
	(No symbol) [0x00E9D04D]
	(No symbol) [0x00ED28B9]
	(No symbol) [0x00ED08CC]
	(No symbol) [0x00ECE9D7]
	(No symbol) [0x00ECD164]
	(No symbol) [0x00EC32A6]
	(No symbol) [0x00EE858C]
	(No symbol) [0x00EC2BFF]
	(No symbol) [0x00EE8804]
	(No symbol) [0x00EFC9EB]
	(No symbol) [0x00EE8386]
	(No symbol) [0x00EC163C]
	(No symbol) [0x00EC269D]
	GetHandleVerifier [0x012B9A22+2655074]
	GetHandleVerifier [0x012ACA24+2601828]
	GetHandleVerifier [0x010C8C0A+619850]
	GetHandleVerifier [0x010C7830+614768]
	(No symbol) [0x00FB05FC]
	(No s